In [ ]:
import numpy as np
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
import loss_funcs
from evaluate import eval_funcs
from algs import *

from utils.orient import OrientUtils
from evaluate.evaluator import Evaluator
from utils.visualize import SearchPlotter
from utils.image import ImageUtils

import mealpy
import cv2 as cv

plotter = SearchPlotter(update_freq=100)

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/mug/world.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/mug/world_sim.xml", cam_config)

loss_func = loss_funcs.IOU()

alg = UniformSampling(sim_viewer, loss_func=loss_func)
alg_config = UniformSampling.Config(time_limit=200, min_samples=350)

eval_func=eval_funcs.XorDiff(0.1)
evaluator = Evaluator(world_viewer, eval_func=eval_func)

init_location = (0, 1.3, 0.3)
random_orientations = OrientUtils.generate_random(80)
eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]

for ref_pos in eval_positions:
    orig_img = world_viewer.get_view_cropped(ref_pos)
    pred_pos, loss = alg.solve(orig_img, ref_pos.location, alg_config)
    pred_img = world_viewer.get_view_cropped(pred_pos)

    cv.imshow("Original", orig_img)
    cv.waitKey(0)
    cv.imshow("Predicted", pred_img)
    cv.waitKey(0)

    orig_depth = world_viewer.get_view_cropped(ref_pos, depth=True)
    pred_depth = world_viewer.get_view_cropped(pred_pos, depth=True)

    eval_loss = eval_func(orig_depth, pred_depth)
    print(eval_loss)

for alg, config in [(alg, alg_config)]:
    eval_losses = evaluator.evaluate(alg, config, eval_positions)
    print(f"{type(alg).__name__}: {eval_losses}")


In [ ]:
import numpy as np
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
import loss_funcs
from evaluate import eval_funcs
from algs import *

from utils.orient import OrientUtils
from evaluate.evaluator import Evaluator
# from utils.visualize import SearchPlotter
from utils.image import ImageUtils

import mealpy
import cv2 as cv

# plotter = SearchPlotter(update_freq=100, alpha=[0.985])

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.3), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/hammer/world.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/hammer/world_sim.xml", cam_config)

loss_func = loss_funcs.IOU()
alg_config = MealAlgorithm.Config(time_limit=15, silent=True)

eval_func=eval_funcs.XorDiff(0.1)

evaluator = Evaluator(world_viewer, eval_func=eval_func)
# evaluator.register_callback(lambda x: plotter.reset())

init_location = (0, 1.3, 0.3)
random_orientations = OrientUtils.generate_uniform(3)
eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]

algorithms: list[MealAlgorithm] = [
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.OriginalSA(temp_init=10 * 2 * np.pi, step_size=0.1)),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.SwarmSA(temp_init=10 * 2 * np.pi, step_size=np.pi/100)),
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.OriginalPSO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.AIW_PSO()),
]
# meal_alg = mealpy.human_based.BRO.DevBRO()
# meal_alg = mealpy.math_based.CEM.OriginalCEM()
# meal_alg = mealpy.music_based.HS.DevHS(pop_size=10)
# meal_alg = mealpy.swarm_based.PSO.OriginalPSO()
# meal_alg = mealpy.physics_based.SA.OriginalSA(pop_size=10, temp_init=10 * 2 * np.pi, step_size=0.2)
# meal_alg = mealpy.human_based.ICA.OriginalICA(empire_count=7, revolution_rate=0.4) # best!!
# meal_alg = mealpy.bio_based.BBO.DevBBO() # potentially good!
meal_alg = mealpy.bio_based.TSA.OriginalTSA() # potentially good!

algorithms: list[MealAlgorithm] = [
    MealAlgorithm(sim_viewer, loss_func, meal_alg),
]

# evaluator.enable_logging("Ealuations/UniformDet/Mug/IOU/")
for alg in algorithms:
    # alg.register_callback(plotter.add_data)
    eval_losses = evaluator.evaluate(alg, alg_config, eval_positions)
    print(f"{alg.get_name()}: {eval_losses}")

world_viewer.close()
sim_viewer.close()
# plotter.close()

In [ ]:
import loss_funcs
from algs import *

import mealpy
from multiprocessing import Process
import evaluation_process as ev

loss_func = loss_funcs.IOU()
alg_config = MealAlgorithm.Config(time_limit=15, silent=True)
root_path = "Ealuations/OverNight/UniformDet/Hammer/IOU/"

if __name__ == '__main__':
    for idx, (alg_name, meal_alg) in enumerate(mealpy.get_all_optimizers().items()):
        if "DevSARO" not in alg_name:
            continue
        print(f"===================== Epoch {idx} =====================")
        print(f"Starting Evaluation of: {alg_name}")
        p = Process(target=ev.evaluate, args=(meal_alg,alg_config,loss_func,root_path))
        p.start()
        p.join()


In [ ]:
import numpy as np
from utils.orient import OrientUtils
len(OrientUtils.generate_uniform(9))

# Analisys

## Plotting Results

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from evaluate.eval_log import MealLog
from mealpy.utils.history import History
from utils.file import LogFiles
import numpy as np

path = "grid_search/"


directories = LogFiles(path, scan_dirs=True)

In [2]:
a = MealLog.load("grid_search/airplane/OriginalICA_20240320-035635.pickle")
print(a.history_list[0].__dict__.keys())
print((a.history_list[0].list_epoch_time))
time = np.cumsum(a.history_list[0].list_epoch_time)

dict_keys(['list_global_best', 'list_current_best', 'list_epoch_time', 'list_global_best_fit', 'list_current_best_fit', 'list_population', 'list_diversity', 'list_exploitation', 'list_exploration', 'list_global_worst', 'list_current_worst', 'epoch', 'log_to', 'log_file', 'logger'])
[2.470320265740156, 2.395192537456751, 2.5515127023682, 2.719743656925857, 2.4360504746437073, 2.6346693970263004]


In [26]:
times = []
for hist in a.history_list:
    # for t in hist.list_epoch_time:
        # times.append(t)
    print(len(hist.list_epoch_time))
    # times.extend(hist.list_epoch_time)
print(len(times))

6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
0


In [19]:

np.array(times, ndmin=2).reshape(a.history_list[0].epoch, -1)

ValueError: cannot reshape array of size 119 into shape (6,newaxis)

In [3]:
a.history_dataframe()

,alg,sample,epoch,eval_loss,list_epoch_time,list_global_best_fit
0,OriginalICA,0,0,0.038339,2.470320,0.456562
1,OriginalICA,0,1,0.038339,2.395193,0.306410
2,OriginalICA,0,2,0.038339,2.551513,0.165242
3,OriginalICA,0,3,0.038339,2.719744,0.122628
4,OriginalICA,0,4,0.038339,2.436050,0.099407
...,...,...,...,...,...,...
114,OriginalICA,19,1,0.037917,2.605211,0.375155
115,OriginalICA,19,2,0.037917,2.410470,0.260627
116,OriginalICA,19,3,0.037917,2.612240,0.207493
117,OriginalICA,19,4,0.037917,2.628524,0.168721


In [ ]:
for key, val in a.history_list[0].__dict__.items():
    print(f"key={key}; type={type(val)}; len={len(val)}")

In [ ]:
a.history_list[0].list_global_best

In [ ]:
# for i in range(len(a.history_list)):
sns.lineplot(x=time, y=a.history_list[0].list_global_best_fit)

In [ ]:
raw_data = []
stat_data = []
for directory in directories:
    files = LogFiles(directory, return_full_path=False)
    for filename in files:
        if "OriginalHC" not in filename:
            continue
        df = files.to_dataframe(add_params=True)
        df = df.assign(object=directories.get_filename())
        raw_data.append(df)

        stat_df = files.eval_stats_dataframe(True)
        stat_df = stat_df.assign(object=directories.get_filename())
        stat_data.append(stat_df)
        


dataframe = pd.concat(raw_data, axis=0, ignore_index=True)
statsframe = pd.concat(stat_data, axis=0, ignore_index=True)

In [ ]:
statsframe.head()


In [ ]:
# category = 'filename'
# categories = pd.unique(dataframe[category])
# medians = {category: [], "median": [], "path": []}
# for cat in categories:
#     mask = dataframe[category] == cat
#     median = dataframe[mask][["eval_loss"]].median()
#     medians[category].append(cat)
#     medians["median"].append(median.item())
#     medians["path"].append(dataframe[mask]["path"].to_list()[0])

In [ ]:
# medians["path"]

In [ ]:
# medians = pd.DataFrame(medians)
statsframe.sort_values(by='median', inplace=True)

In [ ]:
# mask = dataframe['object'] == 'hammer'
# stat_mask = statsframe['object'] == 'hammer'

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size':5})

In [ ]:
# fig, ax = plt.subplots(figsize=(10,40))
g = sns.catplot(
    data=dataframe,
    x="params",
    y="eval_loss",
    hue="object",
    kind="box",
    errorbar="sd",
    aspect=2,
    legend='brief',
    order=statsframe['params'],
    # hue_order=medians[category],
)
g.ax.tick_params(axis="x", rotation=90)
g.ax.set_title(f"PSO GridSearch ordered by median eval loss")

g.ax.autoscale_view()
g.ax.autoscale()
g.figure.tight_layout()


In [ ]:
# print(medians[cat].to_list())

# best = MealLog.load(files.root +'/'+ medians[cat].to_list()[0])


In [ ]:

for idx, file_path in enumerate(statsframe['path'].to_list()):
    root = files.root
    log = MealLog.load(file_path)
    params = log.alg_params
    if params['empire_count'] == 7 and params['revolution_prob'] == 0.4 and params['revolution_rate'] == 0.4:
        print(f"position={idx} - {log.alg_params}")


In [ ]:
from typing import Iterable
import numpy as np
from itertools import product
import cv2 as cv

from view_sampler import *
from algs import *

import loss_funcs
from evaluate import eval_funcs

from evaluate.dataset import Dataset
from evaluate.eval_log import MealLog
from evaluate.evaluator import Evaluator
from evaluate.dataset import Dataset
from utils.visualize import *
from utils.concurrent import TqdmPool, silence_output

from main2 import evaluate

import mealpy


OBJECT_NAMES = ["airplane", "hammer", "mug"]

PARAMS: dict[str, dict[str, list]] = {
    mealpy.swarm_based.PSO.OriginalPSO.__name__: {
        "c1": [1, 2.05, 3],
        "c2": [1, 2.05, 3],
        "w": [0.2, 0.4, 0.8],
    },

    mealpy.human_based.ICA.OriginalICA.__name__: {
        "empire_count": [4, 7, 10],
        "assimilation_coeff": [1.5, 2.5],
        "revolution_prob": np.linspace(0.02, 0.4, 3),
        "revolution_rate": np.linspace(0.05, 0.4, 3),
        "revolution_step_size": np.linspace(0.05, 0.3, 3),
    },
    mealpy.human_based.SARO.OriginalSARO.__name__: {
        "se": [0.3, 0.5, 0.7],
        "mu": [5, 10, 15, 20],
    },

    mealpy.evolutionary_based.DE.OriginalDE.__name__: {"strategy": range(6)},
    mealpy.math_based.PSS.OriginalPSS.__name__: {
        "acceptance_rate": [0.8, 0.9, 0.95],
    },
    mealpy.math_based.HC.OriginalHC.__name__: {"neighbour_size": [50, 200, 700, 950]},
}


OPTIMIZERS = [
    mealpy.swarm_based.PSO.OriginalPSO,
    mealpy.swarm_based.MSA.OriginalMSA,
    mealpy.swarm_based.SCSO.OriginalSCSO,
    mealpy.physics_based.SA.OriginalSA,
    mealpy.physics_based.EVO.OriginalEVO,
    mealpy.physics_based.EFO.DevEFO,
    mealpy.physics_based.EO.ModifiedEO,
    mealpy.human_based.ICA.OriginalICA,
    mealpy.human_based.FBIO.DevFBIO,
    mealpy.human_based.SARO.OriginalSARO,
    mealpy.evolutionary_based.GA.BaseGA,
    mealpy.evolutionary_based.CRO.OCRO,
    mealpy.evolutionary_based.DE.OriginalDE,
    mealpy.math_based.PSS.OriginalPSS,
    mealpy.math_based.SCA.DevSCA,
    mealpy.math_based.HC.OriginalHC,
]


OBJ_LOCATION = (0, 1.3, 0.3)

In [ ]:

exec = TqdmPool(4)

run_config = MealRunConfig(time_limit=15, silent=True, seed=0)

dataset = Dataset.create_random(location=OBJ_LOCATION, num_samples=20, seed=1)

results = []

tasks = []

for optimizer_type in OPTIMIZERS:

    optimizer_params = PARAMS[optimizer_type.__name__]

    for param_config in product(*optimizer_params.values()):
        if len(param_config) == 0:
            continue

        kwargs = {}
        for i, param_name in enumerate(optimizer_params.keys()):
            kwargs[param_name] = param_config[i]

        try:
            optimizer = optimizer_type()
            optimizer.set_parameters(kwargs)

            for obj_name in OBJECT_NAMES:

                task = exec.submit(
                    evaluate,
                    optimizer=optimizer,
                    run_config=run_config,
                    obj_name=obj_name,
                    eval_positions=dataset,
                    log_folder=f"grid_search/{obj_name}",
                )

        except Exception as e:
            print(f"Failed to create optimizer {optimizer_type.__name__} with params {kwargs}")
            print(str(e))

exec.shutdown(wait=True)


In [ ]:
exec.shutdown(wait=True)
